In [ ]:
!git clone https://github.com/EmanueleCeglia/PDF-Metadata-Extractor.git

In [ ]:
%cd PDF-Metadata-Extractor

In [ ]:
!pip install -r requirements.txt

In [ ]:
!apt-get install ghostscript -y

## Extract Dates

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from pdf_dates_finder import PDFDatesFinderSemanticSearch  #old method
from pdf_dates_finder import PDFDatesFinderSpace   #new method
import os
import re
from tqdm import tqdm

Download model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B")
model = AutoModelForCausalLM.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B", device_map="auto", load_in_8bit=True)

In [12]:
def post_process_response_dates(text):
    start_date_pattern = r"Start date:\s*([0-9]+[a-z]*\s+[A-Z][a-z]+\s+[0-9]{4})"
    end_date_pattern = r"End date:\s*([0-9]+[a-z]*\s+[A-Z][a-z]+\s+[0-9]{4})"

    start_date_match = re.search(start_date_pattern, text)
    end_date_match = re.search(end_date_pattern, text)

    result = {}
    if start_date_match and end_date_match:
        result['Start date'] = start_date_match.group(1)
        result['End date'] = end_date_match.group(1)
    else:
        return None

    return result

# second level filter
def find_dates_regex(string):
    pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b|\b\d{1,2} [A-Za-z]+ \d{4}\b|\b\d{1,2}(?:st|nd|rd|th)? [A-Za-z]+ \d{4}\b|\b[A-Za-z]+ \d{1,2}(?:st|nd|rd|th)? \d{4}\b'
    dates = re.findall(pattern, string)
    if len(dates) >= 2:
        return f"Start date: {dates[0]} End date: {dates[1]}"
    elif len(dates) == 1:
        return f"Start date: {dates[0]} End date: None"
    else:
        return "No dates found"

Dates extraction with OLD METHOD

In [ ]:
insurances_folder_path = "/content/PDF-Metadata-Extractor/Insurances"

extracted_dates = {}

for root, dirs, files in os.walk(insurances_folder_path):
    for file_name in tqdm(files):
        full_file_path = os.path.join(root, file_name)

        extraction = PDFDatesFinderSemanticSearch(full_file_path)
        extraction.load_pdf()
        extraction.process_text()
        dates = extraction.find_dates()

        for value in dates.values():
          for phrase in value.values():
            if len(phrase):
              sentence = phrase[0]

              prompt = "Find start date and end date from the following sentence: " + sentence
              inputs = tokenizer(prompt, return_tensors="pt")
              generate_ids = model.generate(inputs.input_ids, max_length=1000)
              response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
              response = post_process_response_dates(response)

              if response:
                extracted_dates[str(file_name)] = response
                print(str(file_name) + ' EXTRACTED!')
              else:
                print(str(file_name) + ' NOT FOUND!')

Dates extraction with NEW METHOD

In [14]:
insurances_folder_path = "/content/PDF-Metadata-Extractor/Insurances"

extracted_dates = {}

for root, dirs, files in os.walk(insurances_folder_path):
    for file_name in tqdm(files):
        full_file_path = os.path.join(root, file_name)

        extractor = PDFDatesFinderSpace(full_file_path)
        pages, tables = extractor.extract_mytext()
        paragraphs = [extractor.identify_paragraphs_space(page) for page in pages]
        #print(paragraphs)

        # keywords filter 1th level
        check_kw = False
        for phrase in paragraphs:
          for sentence in phrase:
            if re.search(r'\bperiod\b', sentence, re.IGNORECASE):
              check_kw = True

        if check_kw:
          # Use a list comprehension with regex to keep only the phrases that contain the word "period" (case-insensitive)
          paragraphs = [sublist for sublist in paragraphs if any(re.search(r'\bperiod\b', phrase, re.IGNORECASE) for phrase in sublist)]

          # Now, further filter each sublist to keep only the phrases that contain the word "period"
          paragraphs = [[phrase for phrase in sublist if re.search(r'\bperiod\b', phrase, re.IGNORECASE)] for sublist in paragraphs]


        responses = []
        for phrase in paragraphs:
          if len(phrase):
            for sentence in phrase:
              sentcence = sentence.replace(',', '')
              prompt = "Find start date and end date from the following sentence: " + sentence
              inputs = tokenizer(prompt, return_tensors="pt")
              generate_ids = model.generate(inputs.input_ids, max_length=200)
              response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
              response = post_process_response_dates(response)

              if response:
                  responses.append(response)
                  print(response)

        if responses:
          print(str(file_name) + ' EXTRACTED!')
          extracted_dates[str(file_name)] = responses
        else:
          # Parse with regex
          for phrase in paragraphs:
            if len(phrase):
              if len(phrase)>1:
                phrase = [' '.join(phrase)]
              for sentence in phrase:
                  sentence = sentence.replace(',','')
                  response = find_dates_regex(sentence)
                  responses.append(response)
        if responses:
          print(str(file_name) + ' EXTRACTED!')
          extracted_dates[str(file_name)] = responses
        else:
          print(str(file_name) + ' NOT FOUND!')

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
100%|██████████| 1/1 [01:01<00:00, 61.14s/it]


policy_39.pdf EXTRACTED!
policy_39.pdf NOT FOUND!


0it [00:00, ?it/s]


In [15]:
extracted_dates

{'policy_39.pdf': ['Start date: 31st March 2022 End date: 31st March 2023']}